# Assignment-0
### Done by: Abhinav Gupta, 20171059

Objectives:
- Kickstarter to the Computer Vision course, Spring 2020
- Getting acquainted with OpenCV
- Playing and experimenting with simple computer vision tasks

Let's first import all the libraries that we'd need. 

In [2]:
import cv2
import numpy as np
import sys
import os
from collections import Counter 
from PIL import Image
from matplotlib import pyplot as plt

## Problem 1: Converting Videos to its constituent Images

- We convert a given video to its constituent images, and save the output to a folder. 
- If you want to use a video that's already stored on your computer, then we use `cv2.VideoCapture(path_to_video)`
- Here we use `cv2.VideoCapture(0)`, so that we can use the computer's webcam to record a video.

- We first create a VideoCapture object, and pass zero as a parameter, indicating that we want to record a video using the webcam. 

In [ ]:
# Create a VideoCapture object 
# 0 for webcam
cap = cv2.VideoCapture(0)
count = 0

if (cap.isOpened()== False): 
  print("Error")

- `cv2.imshow` pops open the webcam recording, and you can dance/sing while you're being recorded.
- `cv2.imwrite` wrties each image frame into the 'frames' directory. 
- The loop continues unless we quit by pressing 'q'

In [ ]:
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  
  if ret == True:
    name = './frames/frame' + str(count) + '.jpg'
    print ('Creating...' + name) 
    cv2.imwrite(name, frame)
    cv2.imshow('Frame', frame)
    count+=1

    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
cap.release()
cv2.destroyAllWindows()

- So finally, we've got ourselves a folder 'frames' which contains the constituent images of the video. 
- For the first question of the assignment, I used a video that was already stored on my machine. 
- For the second part, I changed the parameter of VideoCapture as already explained, and recorded a video in real time. That was fun :D
- Here is the link to all the frames that we get from the video:
https://drive.google.com/drive/folders/1p1iUfXoh8hSsDOgtgJOxOkm9fvPGfZqR?usp=sharing

## Problem-2: Getting a video from a sequence of images

- Now, let's go the other way around! 
- We've already got a sequence of images in 'frames' folder. Let's take all the images in that folder and put them in a list called `images`

In [11]:
image_folder = 'frames'
video_name = 'video.avi'

name1 = './frames/frame' + str(0) + '.jpg'
frame = cv2.imread(name1)
height, width, layers = frame.shape

- For creating a video, we are going to use OpenCV's `VideoWriter` function. 
- One of the parameters needed is the width and height of the images. 
- So let's takes the first image - `image[0]` and compute its width and height. 
- And then instantiate the `video` object using `cv2.VideoWriter`

In [12]:
video = cv2.VideoWriter(video_name, 0, 13, (width,height))
# images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]



- Awesome! Now let's iterate through all the images and append them to the video object created! 

In [13]:
for i in range(0,83):
    finalname = './frames/frame' + str(i) + '.jpg'
    video.write(cv2.imread(finalname))

cv2.destroyAllWindows()
video.release()

- And Voila! We've got ourselves a video, created from using a sequence of images! 
- We can control the 'frames per second' or the fps of the video, which is the third parameter in the `cv2.VideoWriter` function. The function itself takes 4 paramters - video file name, encoding, fps, and the (width & height).
- I have taken fps = 10 for the above case. We can speed up the video by increasing the fps. 
- Here's the link: https://drive.google.com/file/d/1oNvTddCaYrW9GCLv6gYaVyWFmPqEzGS_/view?usp=sharing

## Problem-3: Chroma Keying

- And finally, we come to the most interesting part of the assignment! 
- We implement the technique of 'Chroma Keying'! 
- We use two videos for the same, one as the foreground and one as the background. 
- The foreground video is one with a green screen, and coins are falling from the top. 
https://drive.google.com/file/d/1u_JouXoO0LY6gjZai9psr2w5BjBSosex/view?usp=sharing
- The background video is me, playing the guitar xD 
https://drive.google.com/file/d/1RUhF_nHe27NE9nKMEEyeTAP0zBxLova0/view?usp=sharing



- First, let's break down both the videos into their constituent image frames and store them in folders 'final1' and 'final2'
- This is highly similar to the problem above. 

In [ ]:
f_video = './green.mp4'
b_video = './captain.mov'

f_vidcap = cv2.VideoCapture(f_video)
b_vidcap = cv2.VideoCapture(b_video)

count = 0

- We've got the VideoCapture objects, now let's break them down into frames.

In [ ]:
while f_vidcap.isOpened() and b_vidcap.isOpened(): 
    f_success, f_image = f_vidcap.read()
    if f_success == True:
        name = './final1/' + str(count) + '.jpg'
        cv2.imwrite(name, f_image)
    else:
        break
    
    b_success,b_image = b_vidcap.read()
    if b_success == True:
        name = './final2/' + str(count) + '.jpg'
        cv2.imwrite(name, b_image)
    count+=1;

- Now that we have 467 images of both, foreground and background, let's take any one foreground image and try to retrieve the 'green screen' from it, that is, get the pixel values of the background. 
- The 'color' function takes an image, finds the pixel value that is most frequently occurring in the image and returns it. 
- Since our image has a pretty consistent green background, we can assure that the pixel value it returns would be the one belonging to the green components of the image! 

In [ ]:
def color(image):
    img = cv2.imread(image)
    d = img.shape
    a = []

    for i in range(0, d[0]):
        for j in range(0, d[1]):
            a.append(tuple(np.array(img[i,j,:])))

    f = tuple(a)

    most_common,num_most_common = Counter(f).most_common(1)[0]

    return most_common

In [3]:
check = color('./final1/1.jpg')

In [4]:
print(check)

(5, 251, 15)


- Hence, the pixel values are: (5,251,15) - which is a nice version of green colour! Now I need to take my image and make it the background, thereby replacing the green screen! 
- So we merge the two images - and the green screen is replace by my guitar!
- This is exactly what the 'mergeImage' function below does. It takes 2 images and return the merged one. 

In [5]:
def mergeImage(fg, bg):
    a = check
    d2 = bg.shape
    
    for i in range(0, d2[0]):
        for j in range(0, d2[1]):
            if fg[i,j,2] < a[2]+10 and fg[i,j,2] > a[2]-10 and fg[i,j,1] > a[1]-10 and fg[i,j,1] < a[1]+10 and fg[i,j,0] < a[0]+10 and fg[i,j,0] > a[0]-10:
                fg[i,j] = bg[i,j]
                
    return fg

- Now we've already got our foreground and background images in final1 and final2. Let's iterate through them, take images from them simultaneously and pass them to the merge function. 
- The final merged image is stored in the folder 'final3'

In [ ]:
image1_folder = 'final1'
image2_folder = 'final2'

for i in range(0,467):
    name1 = './final1/' + str(i) + '.jpg'
    name2 = './final2/' + str(i) + '.jpg'
    img1 = cv2.imread(name1)
    img2 = cv2.imread(name2)
    
    img = mergeImage(img1, img2)
    name = './final3/' + str(i) + '.jpg' 
    cv2.imwrite(name, img)
    



- Now all our images are in the 'final3' folder. 
- All that's left now is to take those images and form a video, just like we did in Problem-2! 

- Here's an example of how the merged image looks like: 

![example](./merged-image.png)

- Let;s convert the merged images into a video!

In [14]:
image_folder = 'final3'
video_name = 'video2.avi'

name1 = './final3/' + str(0) + '.jpg'
frame = cv2.imread(name1)
height, width, layers = frame.shape

In [15]:
video = cv2.VideoWriter(video_name, 0, 13, (width,height))
# images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

for i in range(0,73):
    finalname = './final3/' + str(i) + '.jpg'
    video.write(cv2.imread(finalname))

cv2.destroyAllWindows()
video.release()

- Sorry, I'm running out of time, so I've only done it for 73 frames! Here's the link to the video: 
https://drive.google.com/file/d/1S3QMBBUSlD6kBaUzMVKnfFdM71xrIfmk/view?usp=sharing
